# 컴피티션 개인전
- 17시30분 마감
- 평가지표: AUC
- 1~3등에게 스타벅스 쿠폰을 전달

In [1]:
%pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.8 MB/s eta 0:00:00


In [2]:
%pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import optuna

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 경로와 시드값

In [5]:
DATA_PATH  = "/content/drive/MyDrive/the_datas/data/"
SEED = 42

# 데이터 불러오기

In [6]:
train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터
train_tr.shape ,test_tr.shape, train_target.shape, submit.shape

((523105, 7), (441196, 7), (14940, 2), (12225, 2))

In [7]:
train_ft = pd.read_csv(f"{DATA_PATH}train_ft_군집분석_34_김이지.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_ft_군집분석_34_김이지.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 488), (12225, 488))

In [8]:
# display(train.head())
# display(train.info())

- 공통 피처 파일 불러오기

In [9]:
# train_ft = pd.read_csv(f"{DATA_PATH}train_common_3.csv") # 학습 데이터(피처)
# test_ft = pd.read_csv(f"{DATA_PATH}test_common_3.csv") # 테스트 데이터(피처)

# train_ft.shape , test_ft.shape

# 결측치 처리

In [10]:
mask = train_ft.isnull().sum() > 0
train_ft.isnull().sum()[mask]

,0
3회이상count,4214
구매금액표준편차,388
구매금액왜도,840
구매금액첨도,1265


In [11]:
mask = test_ft.isnull().sum() > 0
test_ft.isnull().sum()[mask]

,0
3회이상count,3284
구매금액표준편차,242
구매금액왜도,599
구매금액첨도,954


In [12]:
train_ft["3회이상count"] = train_ft["3회이상count"].fillna(0)
train_ft["구매금액표준편차"] = train_ft["구매금액표준편차"].fillna(0)
train_ft["구매금액왜도"] = train_ft["구매금액왜도"].fillna(0)
train_ft["구매금액첨도"] = train_ft["구매금액첨도"].fillna(0)

test_ft["3회이상count"] = test_ft["3회이상count"].fillna(0)
test_ft["구매금액표준편차"] = test_ft["구매금액표준편차"].fillna(0)
test_ft["구매금액왜도"] = test_ft["구매금액왜도"].fillna(0)
test_ft["구매금액첨도"] = test_ft["구매금액첨도"].fillna(0)

In [13]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

# 특성 공학(Feature Engineering)

- ID 변수 제외

In [14]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 487), (12225, 487))

- 추가 피처 만들어 보기

In [15]:
train_ft

,구매횟수,내점일수,구매주기,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,12시이전구매비율,12시이후_18시이전구매비율,...,브랜드_cluster_pivot_횟수비율24,브랜드_cluster_pivot_횟수비율25,브랜드_cluster_pivot_횟수비율26,브랜드_cluster_pivot_횟수비율27,브랜드_cluster_pivot_횟수비율28,브랜드_cluster_pivot_횟수비율29,브랜드_cluster_pivot_횟수비율30,브랜드_cluster_pivot_횟수비율31,브랜드_cluster_pivot_횟수비율32,브랜드_cluster_pivot_횟수비율33
0,16,11,23,0.050000,0.250000,0.400000,0.300000,3,0.300000,0.450000,...,0.000000,0.0,0.062500,0.0,0.0,0.000000,0.000000,0.0,0.125,0.000000
1,41,21,16,0.357143,0.166667,0.357143,0.119048,3,0.214286,0.476190,...,0.121951,0.0,0.024390,0.0,0.0,0.000000,0.000000,0.0,0.000,0.000000
2,102,56,6,0.464912,0.140351,0.175439,0.219298,0,0.122807,0.728070,...,0.157895,0.0,0.052632,0.0,0.0,0.052632,0.000000,0.0,0.000,0.000000
3,191,92,3,0.379147,0.180095,0.236967,0.203791,3,0.161137,0.616114,...,0.750000,0.1,0.600000,0.0,0.0,0.000000,0.000000,0.0,0.000,0.000000
4,55,27,11,0.112903,0.612903,0.209677,0.064516,4,0.096774,0.741935,...,0.047619,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14935,16,10,24,0.000000,0.578947,0.052632,0.368421,2,0.263158,0.578947,...,0.000000,0.0,0.100000,0.1,0.0,0.000000,0.000000,0.2,0.000,0.000000
14936,8,8,29,0.125000,0.625000,0.250000,0.000000,0,0.000000,0.500000,...,0.000000,0.0,0.019231,0.0,0.0,0.000000,0.000000,0.0,0.000,0.019231
14937,22,12,26,0.208333,0.458333,0.208333,0.125000,1,0.375000,0.541667,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.022222,0.0,0.000,0.000000
14938,31,12,28,0.393939,0.393939,0.212121,0.000000,3,0.151515,0.484848,...,0.050000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000,0.000000


In [16]:
cols = [ col for col in train_ft.columns if col.startswith("중_pivot_횟수비율_") ]

In [17]:
# cols

In [18]:
# train_ft["중분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
# train_ft["중분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
# train_ft["중분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

# test_ft["중분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
# test_ft["중분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
# test_ft["중분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

# train_ft.shape, test_ft.shape

In [19]:
# 최소 구매금액범주화
# 중분류 개수

## Feature Encoding

In [20]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
주구매지점,4
최대구매액_대분류,28
최소구매액_대분류,28


In [21]:
%pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 2.0 MB/s eta 0:00:00


In [22]:
import category_encoders as ce

In [23]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 491), (12225, 491))

In [24]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["최소구매액_대분류"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["최소구매액_대분류"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 519), (12225, 519))

In [25]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["최대구매액_대분류"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["최대구매액_대분류"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 547), (12225, 547))

In [26]:
# enc = ce.count.CountEncoder()
# train_ft["최소구매액_대분류_cnt"] = enc.fit_transform(train_ft[["최소구매액_대분류"]])
# test_ft["최소구매액_대분류_cnt"] = enc.transform(test_ft[["최소구매액_대분류"]])

# train_ft.shape, test_ft.shape

In [27]:
# enc = ce.count.CountEncoder()
# train_ft["최대구매액_대분류_cnt"] = enc.fit_transform(train_ft[["최대구매액_대분류"]])
# test_ft["최대구매액_대분류_cnt"] = enc.transform(test_ft[["최대구매액_대분류"]])

# train_ft.shape, test_ft.shape

- 문자열 피처 삭제

In [28]:
cols

['주구매지점', '최대구매액_대분류', '최소구매액_대분류']

In [29]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 544), (12225, 544))

In [30]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

## Feature Scaling

In [31]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [32]:
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,구매횟수,내점일수,구매주기,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,12시이전구매비율,12시이후_18시이전구매비율,...,최대구매액_대분류_19,최대구매액_대분류_20,최대구매액_대분류_21,최대구매액_대분류_22,최대구매액_대분류_23,최대구매액_대분류_24,최대구매액_대분류_25,최대구매액_대분류_26,최대구매액_대분류_27,최대구매액_대분류_28
0,-0.437126,-0.369867,0.002987,-1.029777,0.001191,0.838272,0.338186,0.109631,0.718557,-0.654150,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
1,0.239394,0.144110,-0.356452,0.323951,-0.390607,0.620171,-0.552996,0.109631,0.269465,-0.542415,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
2,1.890101,1.943028,-0.869935,0.798943,-0.514333,-0.304527,-0.059266,-1.643370,-0.209830,0.532172,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
3,4.298510,3.793345,-1.023980,0.420933,-0.327474,0.008592,-0.135636,0.109631,-0.009001,0.054536,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
4,0.618244,0.452496,-0.613193,-0.752532,1.707410,-0.130285,-0.821561,0.693965,-0.346226,0.591326,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497


# 정답 데이터

In [33]:
target = train_target["target"]
target

,target
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
14935,0.0
14936,0.0
14937,0.0
14938,1.0


# importance 0 제거

In [34]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_ft, target, test_size=0.2, random_state=42)

model = LGBMClassifier(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

feature_importances = model.feature_importances_

feature_importance_df = pd.DataFrame({
    'Feature': train_ft.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

feature_importance_df.head(50)
feature_importance_df[feature_importance_df["Importance"]==0]
rows_to_drop = feature_importance_df[feature_importance_df["Importance"]==0].index
feature_importance_df.drop(rows_to_drop, inplace=True)
cols = feature_importance_df['Feature'].tolist()
train_ft = train_ft[cols]
test_ft = test_ft[cols]
train_ft.shape, test_ft.shape

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4687, number of negative: 7265
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.058208 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 60674
[LightGBM] [Info] Number of data points in the train set: 11952, number of used features: 519
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.392152 -> initscore=-0.438276
[LightGBM] [Info] Start training from score -0.438276


((14940, 350), (12225, 350))

# 모델 튜닝하기

In [35]:
class RF_Objective:
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = KFold(n_splits=5, shuffle=True, random_state=self.seed)

    def __call__(self, trial):
        hp = {
              'n_estimators': trial.suggest_int('n_estimators', 100, 1001),
              'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
              'max_depth': trial.suggest_int('max_depth', 3, 41),
              'min_samples_split': trial.suggest_int('min_samples_split', 6, 30),
              'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
        }
        model = RandomForestClassifier(random_state=self.seed, **hp)
        scores = cross_val_score(model, self.x, self.y, cv=self.cv, scoring='f1_macro', n_jobs=-1)
        return scores.mean()

class XG_Objective:
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = KFold(n_splits=5, shuffle=True, random_state=self.seed)

    def __call__(self, trial):
        hp = {
            'n_estimators': trial.suggest_int('n_estimators', 50,201),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.31, step=0.01),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'gamma': trial.suggest_float('gamma', 0.1, 5.1, step=0.1),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0, step=0.1)
            }
        model = XGBClassifier(random_state=self.seed, **hp)
        scores = cross_val_score(model, self.x, self.y, cv=self.cv, scoring='f1_macro', n_jobs=-1)
        return scores.mean()

class LR_Objective:
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = KFold(n_splits=5, shuffle=True, random_state=self.seed)

    def __call__(self, trial):
        hp = {
            'C': trial.suggest_float('C', 0.001, 10, step=0.001),
            'penalty': trial.suggest_categorical('penalty', ['l2',  None]),
            }
        model = LogisticRegression(random_state=self.seed, **hp)
        scores = cross_val_score(model, self.x, self.y, cv=self.cv, scoring='f1_macro', n_jobs=-1)
        return scores.mean()

class LGB_Objective:
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = KFold(n_splits=5, shuffle=True, random_state=self.seed)

    def __call__(self, trial):
        hp = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 501),
            'learning_rate': trial.suggest_float('learning_rate',0.01, 0.31, step=0.01),
            'max_depth': trial.suggest_int('max_depth', 3, 16),
            'num_leaves': trial.suggest_int('num_leaves', 20,101),
            'min_child_samples': trial.suggest_int('min_child_samples', 5,31),
            }
        model = LGBMClassifier(random_state=self.seed, **hp)
        scores = cross_val_score(model, self.x, self.y, cv=self.cv, scoring='f1_macro', n_jobs=-1)
        return scores.mean()

In [36]:
sampler = optuna.samplers.TPESampler(seed=SEED) # 대체 모델

In [37]:
# rf_study = optuna.create_study(
#     direction='maximize',
#     sampler= sampler
# )

In [38]:
# rf_objective_func = RF_Objective(train_ft, target, SEED)
# rf_study.optimize(rf_objective_func, n_trials=10)

In [39]:
# rf_study.best_value, rf_study.best_trial.params

In [40]:
# lr_study = optuna.create_study(
#     direction='maximize',
#     sampler= sampler
# )

In [41]:
# mask = train_ft.isna().sum() > 0
# train_ft.isna().sum()[mask]

In [42]:
# lr_objective_func = LR_Objective(train_ft, target, SEED)
# lr_study.optimize(lr_objective_func, n_trials=200)

In [43]:
# lr_study.best_value, lr_study.best_trial.params

In [44]:
xg_study = optuna.create_study(
    direction='maximize',
    sampler= sampler
)

[I 2024-11-14 15:19:55,349] A new study created in memory with name: no-name-6d8b4d0b-84e7-42be-ac95-7371b10ee960


In [45]:

xg_objective_func = XG_Objective(train_ft, target, SEED)
xg_study.optimize(xg_objective_func, n_trials=150)

[I 2024-11-14 15:20:30,285] Trial 0 finished with value: 0.6841901868019977 and parameters: {'n_estimators': 106, 'learning_rate': 0.3, 'max_depth': 10, 'gamma': 3.1, 'subsample': 0.5, 'colsample_bytree': 0.5}. Best is trial 0 with value: 0.6841901868019977.
[I 2024-11-14 15:21:04,071] Trial 1 finished with value: 0.6936140977683534 and parameters: {'n_estimators': 58, 'learning_rate': 0.27, 'max_depth': 9, 'gamma': 3.7, 'subsample': 0.5, 'colsample_bytree': 1.0}. Best is trial 1 with value: 0.6936140977683534.
[I 2024-11-14 15:21:36,718] Trial 2 finished with value: 0.7158146677242364 and parameters: {'n_estimators': 176, 'learning_rate': 0.06999999999999999, 'max_depth': 4, 'gamma': 1.0, 'subsample': 0.6, 'colsample_bytree': 0.8}. Best is trial 2 with value: 0.7158146677242364.
[I 2024-11-14 15:22:55,332] Trial 3 finished with value: 0.7086702809934929 and parameters: {'n_estimators': 115, 'learning_rate': 0.09999999999999999, 'max_depth': 9, 'gamma': 0.8, 'subsample': 0.6, 'colsampl

In [46]:
xg_study.best_value, xg_study.best_trial.params

(0.7195512049684474,
 {'n_estimators': 201,
  'learning_rate': 0.09999999999999999,
  'max_depth': 4,
  'gamma': 2.9000000000000004,
  'subsample': 0.6,
  'colsample_bytree': 0.5})

In [47]:
# xg_study.best_trial.params =

In [48]:
lgb_study = optuna.create_study(
    direction='maximize',
    sampler= sampler
)

lgb_objective_func = LGB_Objective(train_ft, target, SEED)
lgb_study.optimize(lgb_objective_func, n_trials=150)

[I 2024-11-14 16:56:43,968] A new study created in memory with name: no-name-b903bd4b-d694-4915-90fd-7e7e2433c6f7
[I 2024-11-14 16:57:44,790] Trial 0 finished with value: 0.7131038376766995 and parameters: {'n_estimators': 225, 'learning_rate': 0.09, 'max_depth': 14, 'num_leaves': 49, 'min_child_samples': 12}. Best is trial 0 with value: 0.7131038376766995.
[I 2024-11-14 16:58:31,505] Trial 1 finished with value: 0.717885901018725 and parameters: {'n_estimators': 295, 'learning_rate': 0.05, 'max_depth': 14, 'num_leaves': 26, 'min_child_samples': 31}. Best is trial 1 with value: 0.717885901018725.
[I 2024-11-14 16:58:56,738] Trial 2 finished with value: 0.7163968981429616 and parameters: {'n_estimators': 399, 'learning_rate': 0.06999999999999999, 'max_depth': 3, 'num_leaves': 86, 'min_child_samples': 24}. Best is trial 1 with value: 0.717885901018725.
[I 2024-11-14 16:59:27,964] Trial 3 finished with value: 0.7053853990229105 and parameters: {'n_estimators': 379, 'learning_rate': 0.2400

In [49]:
lgb_study.best_value, lgb_study.best_trial.params

(0.7199995387154957,
 {'n_estimators': 414,
  'learning_rate': 0.04,
  'max_depth': 14,
  'num_leaves': 33,
  'min_child_samples': 15})

In [50]:
# (0.8977595290050033,
#  {'n_estimators': 422,
#   'learning_rate': 0.03,
#   'max_depth': 12,
#   'num_leaves': 50,
#   'min_child_samples': 30,
#   'subsample': 0.8,
#   'colsample_bytree': 0.5,
#   'reg_alpha': 0.8,
#   'reg_lambda': 1.0})

# cv 점수 확인하기

In [51]:
cv = KFold(5, random_state=SEED, shuffle=True)

In [52]:
models = [
    #('lr', LogisticRegression(random_state=SEED, **lr_study.best_trial.params)),
    #('rf', RandomForestClassifier(random_state=SEED, **rf_study.best_trial.params)),
    ('xg', XGBClassifier(random_state=SEED, **xg_study.best_trial.params)),
    ('lgb', LGBMClassifier(random_state=SEED, **lgb_study.best_trial.params))
    ]

In [53]:
model_score = []

for model_name, model in models:
    scores = cross_val_score(model, train_ft, target, cv=cv, n_jobs=-1, scoring="f1_macro")
    model_score.append((model_name, scores.mean()))

In [54]:
model_score

[('xg', 0.7195512049684474), ('lgb', 0.7199995387154957)]

In [55]:
model_score

[('xg', 0.7195512049684474), ('lgb', 0.7199995387154957)]

# 모델 학습및 테스트 데이터 예측

In [56]:
# from xgboost import XGBClassifier

# model = XGBClassifier(
#     random_state=SEED,
#     n_estimators=193,
#     learning_rate=0.15,
#     max_depth=3,
#     gamma=2.6,
#     subsample=0.7,
#     colsample_bytree=0.6
# )


- xg

In [57]:
model_xg = XGBClassifier(random_state=SEED, **xg_study.best_trial.params)

In [58]:
model_xg.fit(train_ft, target)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=2.9000000000000004, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.09999999999999999,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=201, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [59]:
pred_proba_xg = model_xg.predict_proba(test_ft)[:,1]
pred_proba_xg

array([0.06948423, 0.3641387 , 0.31090063, ..., 0.05142048, 0.05794052,
       0.03510071], dtype=float32)

In [60]:
pred_xg = model_xg.predict(test_ft)
pred_xg

array([0, 0, 0, ..., 0, 0, 0])

In [61]:
test_ft.shape

(12225, 350)

# 저장

In [62]:
submit

,ID,target
0,test_0,0.5
1,test_1,0.5
2,test_2,0.5
3,test_3,0.5
4,test_4,0.5
...,...,...
12220,test_12220,0.5
12221,test_12221,0.5
12222,test_12222,0.5
12223,test_12223,0.5


In [63]:
submit["target"] = pred_proba_xg
submit

,ID,target
0,test_0,0.069484
1,test_1,0.364139
2,test_2,0.310901
3,test_3,0.760561
4,test_4,0.593159
...,...,...
12220,test_12220,0.718816
12221,test_12221,0.336190
12222,test_12222,0.051420
12223,test_12223,0.057941


In [64]:
submit.to_csv(f"{DATA_PATH}submit_optuna_proba_xg_3.csv",index=False)

In [65]:
submit["target"] = pred_xg
submit

,ID,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,1
4,test_4,1
...,...,...
12220,test_12220,1
12221,test_12221,0
12222,test_12222,0
12223,test_12223,0


In [66]:
submit.to_csv(f"{DATA_PATH}submit_optuna_xg_3.csv",index=False)

In [67]:
# pred_xg_train = model_xg.predict(train_ft)
# pred_xg_train

In [68]:
# from sklearn.metrics import f1_score
# f1_score(target, pred_xg_train, average='macro')

- lgb

In [69]:
model_lgb = LGBMClassifier(random_state=SEED, **lgb_study.best_trial.params)

In [70]:
model_lgb.fit(train_ft, target)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.044927 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 57501
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 350
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995


LGBMClassifier(learning_rate=0.04, max_depth=14, min_child_samples=15,
               n_estimators=414, num_leaves=33, random_state=42)

In [71]:
pred_proba_lgb = model_lgb.predict_proba(test_ft)[:,1]
pred_proba_lgb.shape

(12225,)

In [72]:
pred_lgb = model_lgb.predict(test_ft)
pred_lgb

array([0., 0., 0., ..., 0., 0., 0.])

In [73]:
test_ft.shape

(12225, 350)

# 저장

In [74]:
submit

,ID,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,1
4,test_4,1
...,...,...
12220,test_12220,1
12221,test_12221,0
12222,test_12222,0
12223,test_12223,0


In [75]:
submit["target"] = pred_proba_lgb
submit

,ID,target
0,test_0,0.047812
1,test_1,0.306878
2,test_2,0.382988
3,test_3,0.822961
4,test_4,0.602297
...,...,...
12220,test_12220,0.663311
12221,test_12221,0.203951
12222,test_12222,0.059392
12223,test_12223,0.107973


In [76]:
submit.to_csv(f"{DATA_PATH}submit_optuna_proba_lgb_3.csv",index=False)

In [77]:
submit["target"] = pred_lgb
submit

,ID,target
0,test_0,0.0
1,test_1,0.0
2,test_2,0.0
3,test_3,1.0
4,test_4,1.0
...,...,...
12220,test_12220,1.0
12221,test_12221,0.0
12222,test_12222,0.0
12223,test_12223,0.0


In [78]:
submit.to_csv(f"{DATA_PATH}submit_optuna_lgb_3.csv",index=False)

In [79]:
# pred_lgb_train = model_lgb.predict(train_ft)
# pred_lgb_train

In [80]:
# from sklearn.metrics import f1_score
# f1_score(target, pred_lgb_train, average='macro')

In [81]:
#앙상블
pred = pred_proba_xg * 0.5 + pred_proba_lgb * 0.5
pred

array([0.05864806, 0.33550835, 0.34694407, ..., 0.05540623, 0.08295691,
       0.02794713])

In [82]:
pred_final = (pred > 0.5).astype(int)
pred_final.mean()

0.32098159509202456

# 예측 파일 저장후 컴피티션 페이지에 제출

In [83]:
submit

,ID,target
0,test_0,0.0
1,test_1,0.0
2,test_2,0.0
3,test_3,1.0
4,test_4,1.0
...,...,...
12220,test_12220,1.0
12221,test_12221,0.0
12222,test_12222,0.0
12223,test_12223,0.0


In [84]:
submit["target"] = pred
submit

,ID,target
0,test_0,0.058648
1,test_1,0.335508
2,test_2,0.346944
3,test_3,0.791761
4,test_4,0.597728
...,...,...
12220,test_12220,0.691063
12221,test_12221,0.270070
12222,test_12222,0.055406
12223,test_12223,0.082957


#업로드

In [85]:
submit.to_csv(f"{DATA_PATH}submit_새출발_튜닝_5_proba.csv",index=False)

In [86]:
submit['target'] = pred_final
submit

,ID,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,1
4,test_4,1
...,...,...
12220,test_12220,1
12221,test_12221,0
12222,test_12222,0
12223,test_12223,0


In [87]:
submit.to_csv(f"{DATA_PATH}submit_새출발_튜닝_5.csv",index=False)